In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


In [7]:
chip_df = pd.read_csv('automotive_chips_reliability_dataset.csv')


In [10]:
# Check for missing data
chip_df.isnull().sum()

Chip ID                          0
Technology Node                  0
FSDOI Type                       0
Gate Density                     0
Substrate Type                   0
Power Consumption                0
Thermal Dispersion Efficiency    0
Mechanical Stress Resistance     0
Temperature Range                0
Humidity Levels                  0
Vibration Levels                 0
External Load Stress             0
Thermal Stress                   0
Mechanical Stress                0
Electrical Density               0
Electromigration Risk            0
Deformation                      0
Thermal Test Result              0
Mechanical Stress Test Result    0
Failure Mode                     0
Failure Rate (%)                 0
MTTF (hours)                     0
Reliability Score                0
Failure Rate Comparison          0
Statistical Significance         0
dtype: int64

In [11]:
chip_df.dtypes

Chip ID                           object
Technology Node                    int64
FSDOI Type                        object
Gate Density                       int64
Substrate Type                    object
Power Consumption                float64
Thermal Dispersion Efficiency    float64
Mechanical Stress Resistance     float64
Temperature Range                 object
Humidity Levels                    int64
Vibration Levels                  object
External Load Stress               int64
Thermal Stress                   float64
Mechanical Stress                float64
Electrical Density               float64
Electromigration Risk            float64
Deformation                      float64
Thermal Test Result              float64
Mechanical Stress Test Result    float64
Failure Mode                      object
Failure Rate (%)                 float64
MTTF (hours)                       int64
Reliability Score                float64
Failure Rate Comparison           object
Statistical Sign

In [12]:
chip_df['Chip ID'].head()

0    Chip001
1    Chip002
2    Chip003
3    Chip004
4    Chip005
Name: Chip ID, dtype: object